# Sparkify Project Workspace
This workspace contains a tiny subset (128MB) of the full dataset available (12GB). Feel free to use this workspace to build your project, or to explore a smaller subset with Spark before deploying your cluster on the cloud. Instructions for setting up your Spark cluster is included in the last lesson of the Extracurricular Spark Course content.

You can follow the steps below to guide your data analysis and model building portion of this project.

In [2]:
# import libraries
from pyspark.sql import SparkSession, Window
from pyspark.sql.functions import udf, desc, countDistinct,sum
from pyspark.sql.types import IntegerType

import datetime,sys

from pyspark.ml import Pipeline
from pyspark.ml.classification import LogisticRegression,RandomForestClassifier,GBTClassifier
from pyspark.ml.evaluation import MulticlassClassificationEvaluator,BinaryClassificationEvaluator
from pyspark.ml.feature import CountVectorizer, IDF, Normalizer, StandardScaler, StringIndexer, VectorAssembler
from pyspark.ml.regression import LinearRegression
from pyspark.ml.tuning import CrossValidator, ParamGridBuilder

In [3]:
# create a Spark session
spark = SparkSession.builder \
      .master("local").appName("CapProj").getOrCreate()

# Load and Clean Dataset
In this workspace, the mini-dataset file is `mini_sparkify_event_data.json`. Load and clean the dataset, checking for invalid or missing data - for example, records without userids or sessionids. 

In [4]:
#read in data set
file = 'mini_sparkify_event_data.json'
df = spark.read.json(file)

In [5]:
df.printSchema()

root
 |-- artist: string (nullable = true)
 |-- auth: string (nullable = true)
 |-- firstName: string (nullable = true)
 |-- gender: string (nullable = true)
 |-- itemInSession: long (nullable = true)
 |-- lastName: string (nullable = true)
 |-- length: double (nullable = true)
 |-- level: string (nullable = true)
 |-- location: string (nullable = true)
 |-- method: string (nullable = true)
 |-- page: string (nullable = true)
 |-- registration: long (nullable = true)
 |-- sessionId: long (nullable = true)
 |-- song: string (nullable = true)
 |-- status: long (nullable = true)
 |-- ts: long (nullable = true)
 |-- userAgent: string (nullable = true)
 |-- userId: string (nullable = true)



In [6]:
#clean up some null data
#df.count()
df.dropDuplicates()
ndf = df.dropna(how = 'any', subset = ['userId','sessionId']).filter(df.userId != '')
print(df.count(),ndf.count())

286500 278154


# Exploratory Data Analysis
When you're working with the full dataset, perform EDA by loading a small subset of the data and doing basic manipulations within Spark. In this workspace, you are already provided a small subset of data you can explore.

### Define Churn

Once you've done some preliminary analysis, create a column `Churn` to use as the label for your model. I suggest using the `Cancellation Confirmation` events to define your churn, which happen for both paid and free users. As a bonus task, you can also look into the `Downgrade` events.

### Explore Data
Once you've defined churn, perform some exploratory data analysis to observe the behavior for users who stayed vs users who churned. You can start by exploring aggregates on these two groups of users, observing how much of a specific action they experienced per a certain time unit or number of songs played.

In [7]:
#define cancellation event
cancl_event = udf(lambda x : 1 if x=='Cancellation Confirmation' else 0, IntegerType())
ndf = ndf.withColumn('cancl_event', cancl_event('page'))

In [8]:
#Mark the users who cancelled with churn
windowval = Window.partitionBy(ndf.userId).orderBy(ndf.ts.desc()).rangeBetween(Window.unboundedPreceding,0)
ndf = ndf.withColumn('Churn', sum('cancl_event').over(windowval))

Here I convert some columns as part of features

In [9]:
#Get hour from ts field
get_hour = udf(lambda x: int(datetime.datetime.fromtimestamp(x/1000.0).hour), IntegerType())
ndf = ndf.withColumn('Hour', get_hour('ts'))

In [10]:
#index the gender field
indexer = StringIndexer(inputCol="gender", outputCol="indexgender")
ndf = indexer.fit(ndf).transform(ndf)

Now I want to retrieve some information from the event file to be used as 
the part of the features to do modeling. I will retrieve the total number of the 
songs played, the total number of Login/out, the total number of 'Thumbs up'
and total number of 'Thumbs down' in the following:

In [11]:
#Function to retrieve the number of action information from the 'page' column 
def ret_number_val(col, val, ncol, nvalcol, df = ndf ):
    func = udf(lambda x : int (x == val), IntegerType())
    windowval = Window.partitionBy(df.userId).orderBy((df.ts)).rangeBetween(-sys.maxsize,sys.maxsize)
    df = df.withColumn(ncol, func(col))
    df = df.withColumn(nvalcol, sum(ncol).over(windowval))
    return df

In [12]:
#retrieve how many login, songs, thumbup/thumbdown a user had in the time range
vals =['Home','NextSong','Thumbs Up','Thumbs Down']
cols =['onhome','onsong','thumbup','thumbdown']
ncols =['numLogin','numSong','numUp','numDown']
for ix in range(4):
    ndf = ret_number_val('page',vals[ix],cols[ix],ncols[ix],ndf)

In [ ]:
ndf.printSchema()

# Feature Engineering
Once you've familiarized yourself with the data, build out the features you find promising to train your model on. To work with the full dataset, you can follow the following steps.
- Write a script to extract the necessary features from the smaller subset of data
- Ensure that your script is scalable, using the best practices discussed in Lesson 3
- Try your script on the full data set, debugging your script if necessary

If you are working in the classroom workspace, you can just extract features based on the small subset of data contained here. Be sure to transfer over this work to the larger dataset when you work on your Spark cluster.

In [13]:
#Select the feature columns and the label
dfinput = ndf.select(['Hour','itemInSession','status','indexgender','numLogin','numSong','numUp','numDown','Churn'])
dfinput = dfinput.withColumnRenamed('Churn','label')

In [14]:
dfinput.printSchema()

root
 |-- Hour: integer (nullable = true)
 |-- itemInSession: long (nullable = true)
 |-- status: long (nullable = true)
 |-- indexgender: double (nullable = false)
 |-- numLogin: long (nullable = true)
 |-- numSong: long (nullable = true)
 |-- numUp: long (nullable = true)
 |-- numDown: long (nullable = true)
 |-- label: long (nullable = true)



In [15]:
#Assemble the  feature columns to a vector
inputcols = ['Hour','itemInSession','status','indexgender','numLogin','numSong','numUp','numDown']
assembler = VectorAssembler(inputCols = inputcols, outputCol = 'inputVect')
#dfinput = assembler.transform(dfinput)

In [16]:
#Normalize the vector
normaler = Normalizer(inputCol="inputVect", outputCol="inputNormalizer")
#dfinput = normaler.transform(dfinput)

In [17]:
#standardize the vector
scaler = StandardScaler(inputCol='inputVect', outputCol='inputVectStd',withMean=False)
#scalerModel = scaler.fit(dfinput)
#dfinput = scalerModel.transform(dfinput)

# Modeling
Split the full dataset into train, test, and validation sets. Test out several of the machine learning methods you learned. Evaluate the accuracy of the various models, tuning parameters as necessary. Determine your winning model based on test accuracy and report results on the validation set. Since the churned users are a fairly small subset, I suggest using F1 score as the metric to optimize.

In [18]:
#split the dataset into train, test and validation
train,test,validation = dfinput.randomSplit([0.6,0.2,0.2], seed = 42)

In [19]:
#This is the eveluator used for evaluating the following models
evaluator=MulticlassClassificationEvaluator()

Check Logistic Regression model first.

In [19]:
lr = LogisticRegression(featuresCol = 'inputVectStd', labelCol = 'label', regParam=0.0, maxIter=10)
#lrModel = lr.fit(train)

In [20]:
#Generate pipeline and train the model
stagelr = [assembler,scaler,lr]
pipelinelr = Pipeline(stages = stagelr)
pipelrModel=pipelinelr.fit(train)

In [21]:
#predict the test data set and evaluate the result
res = pipelrModel.transform(test)
evaluator.evaluate(res)

0.7625051717384552

In [34]:
#predict the validation data set and evaluate the result
resVal =pipelrModel.transform(validation)
evaluator.evaluate(resVal)

0.7651991856497083

In [22]:
#Parameters to tune the model with crossvalidation
paramGrid = (ParamGridBuilder()
             .addGrid(lr.regParam, [0.0, 1.0])
             .addGrid(lr.maxIter, [10, 20])
             .build())

In [24]:
#select the model with cross validator
cv = CrossValidator(estimator=pipelinelr,
                          estimatorParamMaps=paramGrid,
                          evaluator=MulticlassClassificationEvaluator(),
                          numFolds=3)

cvlrModel = cv.fit(train)

In [38]:
cvlrPred = cvlrModel.transform(test)
evaluator.evaluate(cvlrPred)

In [32]:
cvlrPredVal = cvlrModel.transform(validation)
evaluator.evaluate(cvlrPredVal)

0.7651991856497083

Now check the Random Forest Classification Model.

In [25]:
rf = RandomForestClassifier(featuresCol = 'inputVectStd', labelCol = 'label')
#rfModel= rf.fit(train)

In [26]:
#Generate pipeline and train the RF model
stagerf = [assembler,scaler,rf]
pipelinerf = Pipeline(stages = stagerf)
piperfModel=pipelinerf.fit(train)

In [43]:
#predict the test data set and evaluate the result
res1=piperfModel.transform(test)
evaluator.evaluate(res1)

0.8731752967941295

In [27]:
#Parameters to tune the model
paramGrid = (ParamGridBuilder()
             .addGrid(rf.maxDepth, [2, 6])
             .addGrid(rf.maxBins, [20, 60])
             .addGrid(rf.numTrees, [5, 20])
             .build())

In [28]:
cv = CrossValidator(estimator=pipelinerf,
                          estimatorParamMaps=paramGrid,
                          evaluator=MulticlassClassificationEvaluator(),
                          numFolds=3)
cvrfModel = cv.fit(train)

In [26]:
#predict the test data set and evaluate the result
res = cvrfModel.transform(test)
evaluator.evaluate(res)

0.8795728320203504

In [29]:
#predict the validation data set and evaluate the result
resv = cvrfModel.transform(validation)
evaluator.evaluate(resv)

0.9192969269832294

Now check Gradient Boosted Tree Model: 

In [20]:
gbt = GBTClassifier(featuresCol = 'inputVectStd', labelCol = 'label', maxIter=10)
#gbtModel = gbt.fit(train)

In [21]:
#Generate pipeline and train GBT model
stagegbt = [assembler,scaler,gbt]
pipelinegbt = Pipeline(stages = stagegbt)
pipegbtModel=pipelinegbt.fit(train)

In [20]:
#predict the test data set and evaluate the result
prediction = pipegbtModel.transform(test)
evaluator.evaluate(prediction)

0.9410159572052337

In [22]:
#parameters for tuning the GBT model with crossvalidation
paramGrid = (ParamGridBuilder()
             .addGrid(gbt.maxDepth, [2, 6])
             .addGrid(gbt.maxBins, [20, 50])
             .addGrid(gbt.maxIter, [10, 20])
             .build())

In [ ]:
#Tune model with cross validations.
cv = CrossValidator(estimator=pipelinegbt, estimatorParamMaps=paramGrid, 
                    evaluator=MulticlassClassificationEvaluator(), numFolds=3)

cvModel = cv.fit(train)

In [22]:
#predict the test data set and evaluate the result
predictiont = cvModel.transform(test)
evaluator.evaluate(predictiont)

0.9961226334101195

In [20]:
#predict the validation data set and evaluate the result
predictions = cvModel.transform(validation)
evaluator.evaluate(predictions)

0.9956187444774144

# Final Steps
Clean up your code, adding comments and renaming variables to make the code easier to read and maintain. Refer to the Spark Project Overview page and Data Scientist Capstone Project Rubric to make sure you are including all components of the capstone project and meet all expectations. Remember, this includes thorough documentation in a README file in a Github repository, as well as a web app or blog post.

In [ ]:
!date